## Gift Contract Testing

In [1]:
from web3.auto import w3
import pandas as pd
from cyberpy import generate_wallet, address_to_address
from cyberpy._message_signer import Message
from cyberpy._wallet import seed_to_privkey
from eth_account.messages import encode_defunct
import json
from subprocess import Popen, PIPE
import os
from time import time
import ipfshttpclient
from tqdm.notebook import tqdm
from base64 import b64encode

client = ipfshttpclient.connect()

NUMBER_OF_PARTICIPANTS = 100
KEY_PHRASE = 'KEY PHRASE'
NICKNAME_LIST = [f'john{round(time())}{number}' for number in range(NUMBER_OF_PARTICIPANTS)]
AVATAR_CID_LIST = [client.add_str(_nickname + 'avatar') for _nickname in NICKNAME_LIST]
CLAIM_AMOUNT_LIST = [1_000_000] * NUMBER_OF_PARTICIPANTS

INITIAL_BALANCE = str(1_000_000_000)
COEF_UP = str(20)
COEF_DOWN = str(5)
TARGET_CLAIM = str(2)

WALLET_ADDRESS = os.getenv('WALLET_ADDRESS')
NODE_URL = 'https://rpc.space-pussy-1.cybernode.ai:443'
NETWORK = 'space-pussy-1'

INIT_SUBSPACE_CONTRACTS = False
SUBSPACE_CODE_ID = str(21)
NAME_SUBSPACE_CONTRACT_ADDRESS = 'bostrom19zmt9dlz3wzu6jqgr9409el7nvxw66u8n67ta5424kwcg0qf269s6t03zt'
AVATAR_SUBSPACE_CONTRACT_ADDRESS = 'bostrom16ulvzd0xp9g6wdhnnhz584ed8xzxm5wkmrsznnzk7qsskegkfutq5djtqs'
PROOF_SUBSPACE_CONTRACT_ADDRESS = 'bostrom1ghwwe2nhxjfy7qulgu6zzmghmnjj3rx8xldr8aytghayxlftz6aqxy4c7k'

INIT_PASSPORT_CONTRACT = False
PASSPORT_CODE_ID = str(25)
PASSPORT_CONTRACT_ADDRESS = 'bostrom15hzg7eaxgs6ecn46gmu4juc9tau2w45l9cnf8n0797nmmtkdv7jscv88ra'

INIT_GIFT_CONTRACT = True
GIFT_CODE_ID = str(20)
GIFT_CONTRACT_ADDRESS = 'bostrom1rt2acjyhs4jfjdq56pftpu7762hy9gfl63je6fnhwrc5p5y4kmuqxg0262'

ROOT_SOURCE_FILE_NAME = 'root_testing_source_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
ROOT_FILE_NAME = 'root_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses'


def execute_bash(bash_command: str, display_data: bool = False) -> [str, str]:
    _output, _error = Popen(bash_command, shell=True, stdout=PIPE, stderr=PIPE).communicate()
    if _error:
        display_data = True
    if display_data:
        print(bash_command)
        if _output:
            try:
                print(json.dumps(json.loads(_output.decode('utf-8')), indent=4, sort_keys=True))
            except json.JSONDecodeError:
                print(_output)
        if _error:
            print(_error)
    return _output.decode('utf-8'), _error.decode('utf-8')

/Users/sergenedashkovsky/Library/Python/3.8/lib/python/site-packages/ipfshttpclient/client/__init__.py:75: VersionMismatch: Unsupported daemon version '0.10.0' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


### Generate addresses and sign messages

In [2]:
claims_list = []
for i in range(NUMBER_OF_PARTICIPANTS):
    claim_item = {'amount': CLAIM_AMOUNT_LIST[i],
                  'nickname': NICKNAME_LIST[i],
                  'avatar': AVATAR_CID_LIST[i]}

    # Generate Ethereum wallet
    ethereum_wallet = w3.eth.account.create(KEY_PHRASE)
    claim_item['ethereum_address'] = ethereum_wallet.address.lower()
    claim_item['ethereum_private_key'] = ethereum_wallet.privateKey.hex()

    # Generate Bostrom and Cosmos wallet
    bostrom_wallet = generate_wallet()
    claim_item['bostrom_address'] = bostrom_wallet['address']
    claim_item['cosmos_address'] = address_to_address(bostrom_wallet['address'], 'cosmos')
    claim_item['cosmos_seed'] = bostrom_wallet['seed']

    # Create message
    # message_json = {
    #     "nickname": claim_item['nickname'],
    #     "gift_claiming_address": claim_item['ethereum_address'],
    #     "gift_amount": claim_item['amount'],
    #     "target_address": claim_item['bostrom_address']
    # }
    # claim_item['message'] = json.dumps(message_json)
    claim_item['message'] = f"{WALLET_ADDRESS}:QmRX8qYgeZoYM3M5zzQaWEpVFdpin6FvVXvp6RPQK3oufV"

    # Sign message form Ethereum address
    ethereum_signed_message = \
        w3.eth.account.sign_message(
            signable_message=encode_defunct(text=claim_item['message']),
            private_key=ethereum_wallet.privateKey)
    # print(signed_message.signature)
    claim_item['ethereum_message_signature'] = ethereum_signed_message.signature.hex()

    # Sign message form Cosmos address
    cosmos_msg = Message(privkey=seed_to_privkey(seed=claim_item['cosmos_seed']))
    cosmos_msg.add_message(signing_message=claim_item['message'], signer_prefix='cosmos')
    claim_item['cosmos_message_signed_row'] = cosmos_msg.get_signed_message()
    claim_item['cosmos_message_signature'] = b64encode(json.dumps(claim_item['cosmos_message_signed_row']).replace('\n', '').replace(' ', '').encode('utf-8')).decode("utf-8")

    # Verify message
    assert claim_item['ethereum_address'] == w3.eth.account.recover_message(
        signable_message=encode_defunct(text=claim_item['message']),
        signature=claim_item['ethereum_message_signature']).lower()

    claims_list.append(claim_item)

claims_df = pd.DataFrame(claims_list)
claims_df.to_csv('claims_ethereum_test_data_without_proof' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_df

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature
0,1000000,john16456890120,QmUPBakJj5Y3bnFYpAi58UssjnFLyxDVx6CpX8t8cZ3Xce,0xace4ce3820cbc4d08106c6077098efff9c102153,0xcbcfa4354ddfe67d6a0549289a28690a29e2b5cd79f9...,bostrom19vjmuzuhv7wpgceaqxtyyjlejnv9ytek2fhfrd,cosmos19vjmuzuhv7wpgceaqxtyyjlejnv9ytekf6r6a2,scan then essay quality elephant curtain cage ...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0xd265b36b2a8bf1651ecc41b1e8814eff629ed035c9d9...,{'pub_key': 'AzcFxx7ohDBkXQNaLt55aX9AIb+ghOxHd...,eyJwdWJfa2V5IjoiQXpjRnh4N29oREJrWFFOYUx0NTVhWD...
1,1000000,john16456890121,QmRSpsrrsCCfcRfKqbQFfF7X7UdsQqkf9g9y7xckXqVrcs,0x2561dc7aa1f2993b71ebcd42eb7b6d89aea82999,0x2dc1391c2e543012159153af30088206945058295693...,bostrom1vwdnx8ru6s2yxk60m66fc5kzgywvwqdc57jt23,cosmos1vwdnx8ru6s2yxk60m66fc5kzgywvwqdchdxc5k,team visa tent boring claim melt transfer appe...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0x6d65007f9ae5cb02b65283e11aac464266be28b84193...,{'pub_key': 'A+BhdQhp/pqiDzdVq0OE28T32xvWSFNTX...,eyJwdWJfa2V5IjoiQStCaGRRaHAvcHFpRHpkVnEwT0UyOF...
2,1000000,john16456890122,Qmej4fH3ToSYFeDgTQVfaqJBLpP7CR8rpzuNciXeZdec8e,0x943cec118f21b326ff10c716b8ae14c4a2a82b1b,0xca3c4b50d40acb3030a30d0ff21e8f12aeb696786bef...,bostrom1czds378wvgf33qts5lzkvcv6xekf47j7u79pg0,cosmos1czds378wvgf33qts5lzkvcv6xekf47j7ld3jkg,tattoo verify pioneer wild cactus marriage inh...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0x735ddd89ce39a9f85d4a3bfc2332528283436812f648...,{'pub_key': 'A/uZetQ/nscjtDQ5LZcJL5ytHIg03Z9l0...,eyJwdWJfa2V5IjoiQS91WmV0US9uc2NqdERRNUxaY0pMNX...
3,1000000,john16456890123,QmdLhvd7efuCXvCVrECXjGw1PqzgTz9dDkHPZLFwgKNQad,0x37c896cf9ff9be029992ee160b0dae49c07495d4,0x94475aabb305ac4a01c6ee9dd28f302646317ca431bf...,bostrom1dkynfk6lwfts9krzl5lz2ytjqq4kcj5hnah4gp,cosmos1dkynfk6lwfts9krzl5lz2ytjqq4kcj5hswrxkx,subway crawl other foot whale awkward olympic ...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0xe0229274c0e7f50a9860986c8dc25a41e612f16c8ff9...,{'pub_key': 'AjiCYjaC6+TJve2QRIqjYJGwb7UaQuFDs...,eyJwdWJfa2V5IjoiQWppQ1lqYUM2K1RKdmUyUVJJcWpZSk...
4,1000000,john16456890124,QmbAGMrNmdrdrLRJTGReuoWFQKKQ4cPydNGGUgAiJz2PNY,0xa59769e19a327d2d28bddfc2c057195cf32e5b33,0xe01ff9b4d81bb31a9353bca3c918137171ac2a8fc146...,bostrom1ajg7t4zcmu9x9h299nsfct3h6m4mx2k3zx4m0s,cosmos1ajg7t4zcmu9x9h299nsfct3h6m4mx2k3p4pg3h,sock rough away buyer mask vocal position chao...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0x329b4afe2546d797f01a12b9ea29e40de28af96b621a...,{'pub_key': 'AsTg0VX5VcIDyaiAh8YOjHwgPk+3PDB7V...,eyJwdWJfa2V5IjoiQXNUZzBWWDVWY0lEeWFpQWg4WU9qSH...
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1000000,john164568901295,QmQCZk9tBmQ9UvyLcvmB9g9z3gtoMNXdQADnyNJWX94aH4,0x65ec5436573364862ae6a950aa977b41487e5715,0xbca87ea9f0d1e9c0f0fd594758f5d67151d39f280e67...,bostrom15pc0j3j0he3mdepjqg73g6fa0l3nau7jqycrfm,cosmos15pc0j3j0he3mdepjqg73g6fa0l3nau7jrhvshu,record what garbage crumble radar bachelor glu...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0x038be813f5f2d71b24e4092f662019eacc6dca6c0ddf...,{'pub_key': 'A1CHZ152GpbcDpzJ3TvOquL+WPkqdZ9Wq...,eyJwdWJfa2V5IjoiQTFDSFoxNTJHcGJjRHB6SjNUdk9xdU...
96,1000000,john164568901296,QmSTGHiCvHsBP5eUQ8Atbqt8RM9VXc8BewbmcTEm6hV6YT,0x0b1c0f128d7fe50eca0af4be0f83cf775a109330,0x2289c89880cf85377c3f702a56e9498553e43fb566f1...,bostrom1fgfkqjtz4p6lnz9ywnpr7fm09nxlpeemnzstw2,cosmos1fgfkqjtz4p6lnz9ywnpr7fm09nxlpeems3ycsd,minimum zero symptom era dilemma burden luxury...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0xade32187d4825afd435d06a1de72c097abc6878026fb...,{'pub_key': 'AmkHIqXGx93glYtD/kFyeiBYDTIDZOGaU...,eyJwdWJfa2V5IjoiQW1rSElxWEd4OTNnbFl0RC9rRnllaU...
97,1000000,john164568901297,QmRoLrGR4nc7pHixeEan5ZvTQxaJ652RNoiGkMqPMtEYH3,0xba288b21a6759c6785d922dd6224df9f295aec9d,0x6cb6a8659aff2cfc4cf7476e48f9177f0ecb9028f6f9...,bostrom15eyq9h8jldnu2g

## Instantiate Contracts

In [3]:
def instantiate_contract(init_query: str, contract_code_id: str, contract_label: str, amount: str = '', display_data: bool = False) -> str:
    _init_output, _init_error = execute_bash(
        f'''INIT='{init_query}' \
            && cyber tx wasm instantiate {contract_code_id} "$INIT" --from $WALLET {'--amount ' + amount + 'boot' if amount else ''} --label "{contract_label}" \
            -y --gas 3500000 --broadcast-mode block -o json --chain-id={NETWORK} --node={NODE_URL}''')
    if display_data:
        try:
            print(json.dumps(json.loads(_init_output.decode('utf-8')), indent=4, sort_keys=True))
        except json.JSONDecodeError:
            print(_init_output)
    if _init_error:
        print(_init_error)
    _init_json = json.loads(_init_output)
    return [event['attributes'][0]['value'] for event in _init_json['logs'][0]['events'] if event['type'] == 'instantiate'][0]

### Instantiate Subspace Contracts

In [4]:
if INIT_SUBSPACE_CONTRACTS:
    name_subspace_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBSPACE_CODE_ID,
            contract_label='test name subspace')
    avatar_subspace_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBSPACE_CODE_ID,
            contract_label='test avatar subspace')
    proof_subspace_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBSPACE_CODE_ID,
            contract_label='test proof subspace')
else:
    name_subspace_contract_address = NAME_SUBSPACE_CONTRACT_ADDRESS
    avatar_subspace_contract_address = AVATAR_SUBSPACE_CONTRACT_ADDRESS
    proof_subspace_contract_address = PROOF_SUBSPACE_CONTRACT_ADDRESS
print(f'Name subspace contract address: {name_subspace_contract_address}\n'
      f'Avatar subspace contract address: {avatar_subspace_contract_address}\n'
      f'Proof subspace contract address: {proof_subspace_contract_address}')

Name subspace contract address: bostrom19zmt9dlz3wzu6jqgr9409el7nvxw66u8n67ta5424kwcg0qf269s6t03zt
Avatar subspace contract address: bostrom16ulvzd0xp9g6wdhnnhz584ed8xzxm5wkmrsznnzk7qsskegkfutq5djtqs
Proof subspace contract address: bostrom1ghwwe2nhxjfy7qulgu6zzmghmnjj3rx8xldr8aytghayxlftz6aqxy4c7k


### Instantiate Passport Contract

In [5]:
if INIT_PASSPORT_CONTRACT:
    passport_contract_address = \
        instantiate_contract(
            init_query=f'''{{"name":"CPT", "minter":"{WALLET_ADDRESS}", "owner":"{WALLET_ADDRESS}", "symbol":"CPT", "avatar_subspace": "{avatar_subspace_contract_address}", "name_subspace": "{name_subspace_contract_address}", "proof_subspace": "{proof_subspace_contract_address}"}}''',
            contract_code_id=PASSPORT_CODE_ID,
            contract_label='test passport')
else:
    passport_contract_address = PASSPORT_CONTRACT_ADDRESS
print(f'Passport contract address: {passport_contract_address}')

Passport contract address: bostrom15hzg7eaxgs6ecn46gmu4juc9tau2w45l9cnf8n0797nmmtkdv7jscv88ra


### Set executor in the Subspace Contracts

In [6]:
def set_executor_subspace(subspace_contract_address: str, new_executor_address: str, display_data: bool = False):
    return execute_bash(
                f'''REGISTER='{{"update_executer":{{"new_executer":"{new_executor_address}"}}}}' \
                    && CONTRACT="{subspace_contract_address}" \
                    && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET -o json --broadcast-mode block -y --gas=600000 --chain-id={NETWORK} --node={NODE_URL}''',
                display_data=display_data)

if INIT_PASSPORT_CONTRACT or INIT_SUBSPACE_CONTRACTS:
    set_executor_subspace(subspace_contract_address=name_subspace_contract_address, new_executor_address=passport_contract_address)
    set_executor_subspace(subspace_contract_address=avatar_subspace_contract_address, new_executor_address=passport_contract_address)
    set_executor_subspace(subspace_contract_address=proof_subspace_contract_address, new_executor_address=passport_contract_address)

### Instantiate Gift Contract

In [7]:
if INIT_GIFT_CONTRACT:
    gift_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "passport":"{passport_contract_address}", "allowed_native":"boot", "initial_balance":"{INITIAL_BALANCE}", "coefficient_up":"{COEF_UP}", "coefficient_down":"{COEF_DOWN}", "coefficient":"{COEF_UP}", "target_claim":"{TARGET_CLAIM}"}}''',
            contract_code_id=GIFT_CODE_ID,
            amount=INITIAL_BALANCE,
            contract_label='test gift')
else:
    gift_contract_address = GIFT_CONTRACT_ADDRESS
print(f'Gift contract address: {gift_contract_address}')

Gift contract address: bostrom1ky7s3pwgx55dh52h80glk8fyq6k6y5qmrsnpasurxjeqnhyhwh0qfx4jqj


In [8]:
def get_contract_name(contract_address: str) -> str:
    address_dict = {gift_contract_address: 'Gift Contract',
                    passport_contract_address: 'Passport Contract',
                    WALLET_ADDRESS: 'Wallet',
                    name_subspace_contract_address: 'Name Subspace Contract',
                    avatar_subspace_contract_address: 'Avatar Subspace Contract',
                    proof_subspace_contract_address: 'Proof Subspace Contract'}
    try:
        return address_dict[contract_address]
    except KeyError:
        return contract_address


def parse_contract_execution_json(contract_execution_json: str) -> None:
    if len(json.loads(contract_execution_json)['logs']) == 0:
        print(json.loads(contract_execution_json)['raw_log'])
    for log_item in json.loads(contract_execution_json)['logs']:
        for event_item in log_item['events']:
            print('')
            if event_item['type'] == 'message':
                if len(event_item["attributes"]) == 3:
                    print(f'message from {get_contract_name(event_item["attributes"][-1]["value"])} {event_item["attributes"][1]["value"]} {event_item["attributes"][0]["value"]}')
                else:
                    print(event_item)
            elif event_item['type'] == 'execute':
                print('execute')
                for attr_item in event_item["attributes"]:
                    if attr_item["key"] == '_contract_address':
                        print(f'\texecute contract: {get_contract_name(attr_item["value"])}')
                    else:
                        print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'reply':
                print('reply')
                for attr_item in event_item["attributes"]:
                    if attr_item["key"] == '_contract_address':
                        print(f'\treply contract: {get_contract_name(attr_item["value"])}')
                    else:
                        print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'cyberlink':
                print('cyberlinks')
                for i, attr_item in enumerate(event_item['attributes']):
                    if attr_item['key'] == 'particleFrom':
                        print(f'\t{attr_item["value"]} -> {event_item["attributes"][i + 1]["value"]}')
                    elif attr_item['key'] == 'particleTo':
                        pass
                    elif attr_item['key'] == 'neuron':
                        print(f'\tneuron: {get_contract_name(event_item["attributes"][-1]["value"])}\n')
                    else:
                        print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'coin_received':
                print('coin received')
                for attr_item in event_item["attributes"]:
                    print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'coin_spent':
                print('coin spent')
                for attr_item in event_item["attributes"]:
                    print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'wasm':
                print('wasm')
                for attr_item in event_item["attributes"]:
                    print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'transfer':
                print('transfer')
                for attr_item in event_item["attributes"]:
                    print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            else:
                print(event_item)

### Create Merkle Root

In [9]:
root_source_list = [{'address': _item['ethereum_address'],
                     'amount': str(_item['amount'])} for _item in claims_list]+ \
                    [{'address': _item['cosmos_address'],
                      'amount': str(_item['amount'])} for _item in claims_list]

with open(ROOT_SOURCE_FILE_NAME, 'w') as outfile:
    outfile.write(str(root_source_list).replace("'", '"'))

In [10]:
execute_bash(f'rm {ROOT_FILE_NAME}')
root_output, root_error = execute_bash(
    f'merkle-airdrop-cli generateRoot --file {ROOT_SOURCE_FILE_NAME} >> {ROOT_FILE_NAME} && cat {ROOT_FILE_NAME}')
root = root_output.replace('\n', '')
print(f'Merkle root: {root}')

Merkle root: 2b3656e620551269a4eb38345a6b24123d41e3b085c616c579bacb09a350f816


In [11]:
root_register_output, root_register_error = \
    execute_bash(
        f'''REGISTER='{{"register_merkle_root":{{"merkle_root":"{root}"}}}}' \
            && CONTRACT="{gift_contract_address}" \
            && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET --broadcast-mode block -o json -y --chain-id={NETWORK} --node={NODE_URL}''')
parse_contract_execution_json(root_register_output)


execute
	execute contract: Gift Contract

message from Wallet wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Gift Contract
	action: register_merkle_root
	merkle_root: 2b3656e620551269a4eb38345a6b24123d41e3b085c616c579bacb09a350f816


#### Get Merkle Root form the Gift Contract

In [12]:
print(
    execute_bash(
        f'''QUERY='{{"merkle_root": {{}}}}' \
            && cyber query wasm contract-state smart {gift_contract_address} "$QUERY" -o json --chain-id={NETWORK} --node={NODE_URL}''')[0]
)

{"data":{"merkle_root":"2b3656e620551269a4eb38345a6b24123d41e3b085c616c579bacb09a350f816"}}



In [13]:
def generate_proof(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    proof_output, _ = execute_bash(f'merkle-airdrop-cli generateProofs --file {ROOT_SOURCE_FILE_NAME} --address {claim_row[network + "_address"]} --amount {claim_row.amount}',
                                   display_data=display_data)
    return proof_output.replace('\n','').replace(" ",'').replace("'", '"')

claims_df.loc[:, 'ethereum_proof'] = claims_df.apply(lambda x: generate_proof(x), axis=1)
claims_df.loc[:, 'cosmos_proof'] = claims_df.apply(lambda x: generate_proof(x, network='cosmos'), axis=1)
claims_df.to_csv('claims_ethereum_test_data_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_df

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature,ethereum_proof,cosmos_proof
0,1000000,john16456890120,QmUPBakJj5Y3bnFYpAi58UssjnFLyxDVx6CpX8t8cZ3Xce,0xace4ce3820cbc4d08106c6077098efff9c102153,0xcbcfa4354ddfe67d6a0549289a28690a29e2b5cd79f9...,bostrom19vjmuzuhv7wpgceaqxtyyjlejnv9ytek2fhfrd,cosmos19vjmuzuhv7wpgceaqxtyyjlejnv9ytekf6r6a2,scan then essay quality elephant curtain cage ...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0xd265b36b2a8bf1651ecc41b1e8814eff629ed035c9d9...,{'pub_key': 'AzcFxx7ohDBkXQNaLt55aX9AIb+ghOxHd...,eyJwdWJfa2V5IjoiQXpjRnh4N29oREJrWFFOYUx0NTVhWD...,"[""90f4df66fab48f071e17043aefb87a7b15f9a40208aa...","[""cde0210f247f5c99c2e5c2ee0259646f4015bc93a4dc..."
1,1000000,john16456890121,QmRSpsrrsCCfcRfKqbQFfF7X7UdsQqkf9g9y7xckXqVrcs,0x2561dc7aa1f2993b71ebcd42eb7b6d89aea82999,0x2dc1391c2e543012159153af30088206945058295693...,bostrom1vwdnx8ru6s2yxk60m66fc5kzgywvwqdc57jt23,cosmos1vwdnx8ru6s2yxk60m66fc5kzgywvwqdchdxc5k,team visa tent boring claim melt transfer appe...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0x6d65007f9ae5cb02b65283e11aac464266be28b84193...,{'pub_key': 'A+BhdQhp/pqiDzdVq0OE28T32xvWSFNTX...,eyJwdWJfa2V5IjoiQStCaGRRaHAvcHFpRHpkVnEwT0UyOF...,"[""799ec8d507710209e710e28bc404fb5a76c0b7f771b6...","[""5671d8f72f465100aa45c0758226e8417470a3a67ee0..."
2,1000000,john16456890122,Qmej4fH3ToSYFeDgTQVfaqJBLpP7CR8rpzuNciXeZdec8e,0x943cec118f21b326ff10c716b8ae14c4a2a82b1b,0xca3c4b50d40acb3030a30d0ff21e8f12aeb696786bef...,bostrom1czds378wvgf33qts5lzkvcv6xekf47j7u79pg0,cosmos1czds378wvgf33qts5lzkvcv6xekf47j7ld3jkg,tattoo verify pioneer wild cactus marriage inh...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0x735ddd89ce39a9f85d4a3bfc2332528283436812f648...,{'pub_key': 'A/uZetQ/nscjtDQ5LZcJL5ytHIg03Z9l0...,eyJwdWJfa2V5IjoiQS91WmV0US9uc2NqdERRNUxaY0pMNX...,"[""e3dd964da7a4a9253f6550c97064847288fb0b4f099a...","[""533300fcaf70358423b8becd2a085590a22818e2e23b..."
3,1000000,john16456890123,QmdLhvd7efuCXvCVrECXjGw1PqzgTz9dDkHPZLFwgKNQad,0x37c896cf9ff9be029992ee160b0dae49c07495d4,0x94475aabb305ac4a01c6ee9dd28f302646317ca431bf...,bostrom1dkynfk6lwfts9krzl5lz2ytjqq4kcj5hnah4gp,cosmos1dkynfk6lwfts9krzl5lz2ytjqq4kcj5hswrxkx,subway crawl other foot whale awkward olympic ...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0xe0229274c0e7f50a9860986c8dc25a41e612f16c8ff9...,{'pub_key': 'AjiCYjaC6+TJve2QRIqjYJGwb7UaQuFDs...,eyJwdWJfa2V5IjoiQWppQ1lqYUM2K1RKdmUyUVJJcWpZSk...,"[""8230b2ce1377c942876595055d8ac660480637b3e50c...","[""ffde428c35fcc8981b82fb3dd6f74fdf7f856cd473df..."
4,1000000,john16456890124,QmbAGMrNmdrdrLRJTGReuoWFQKKQ4cPydNGGUgAiJz2PNY,0xa59769e19a327d2d28bddfc2c057195cf32e5b33,0xe01ff9b4d81bb31a9353bca3c918137171ac2a8fc146...,bostrom1ajg7t4zcmu9x9h299nsfct3h6m4mx2k3zx4m0s,cosmos1ajg7t4zcmu9x9h299nsfct3h6m4mx2k3p4pg3h,sock rough away buyer mask vocal position chao...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0x329b4afe2546d797f01a12b9ea29e40de28af96b621a...,{'pub_key': 'AsTg0VX5VcIDyaiAh8YOjHwgPk+3PDB7V...,eyJwdWJfa2V5IjoiQXNUZzBWWDVWY0lEeWFpQWg4WU9qSH...,"[""2c370916b8eefae8dcb5af3d5637ec9b9d2d93c7147c...","[""18e1fe97572c09e2ce39067b5c777c0d03e651480d48..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1000000,john164568901295,QmQCZk9tBmQ9UvyLcvmB9g9z3gtoMNXdQADnyNJWX94aH4,0x65ec5436573364862ae6a950aa977b41487e5715,0xbca87ea9f0d1e9c0f0fd594758f5d67151d39f280e67...,bostrom15pc0j3j0he3mdepjqg73g6fa0l3nau7jqycrfm,cosmos15pc0j3j0he3mdepjqg73g6fa0l3nau7jrhvshu,record what garbage crumble radar bachelor glu...,bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz...,0x038be813f5f2d71b24e4092f662019eacc6dca6c0ddf...,{'pub_key': 'A1CHZ152GpbcDpzJ3TvOquL+WPkqdZ9Wq...,eyJwdWJfa2V5IjoiQTFDSFoxNTJHcGJjRHB6SjNUdk9xdU...,"[""f7cc1d46c9c2731ae3592e4f3725a12ea174d8748b91...","[""9e5555d32a25b69e914ff4a3236977d7f97297ca63c9..."
96,1000000,john164568901296,Q

### Create Passports

In [13]:
def create_passport(claim_row: pd.Series, display_data: bool = False):
    return execute_bash(
                f'''REGISTER='{{"create_passport":{{"avatar":"{claim_row["avatar"]}", "nickname":"{claim_row["nickname"]}"}}}}' \
                    && CONTRACT="{passport_contract_address}" \
                    && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET -o json --broadcast-mode block -y --gas=600000 --chain-id={NETWORK} --node={NODE_URL}''',
                display_data=display_data)

for index, row in tqdm(claims_df.iterrows()):
    if index == 0:
        create_passport_json, _ = create_passport(row, display_data=True)
    else:
        create_passport(row)

0it [00:00, ?it/s]

REGISTER='{"create_passport":{"avatar":"QmRge6NNuXmaafJKBC6jcHJ5AhTyiAeyHAfRoTuaidNJvU", "nickname":"john16456888760"}}'                     && CONTRACT="bostrom15hzg7eaxgs6ecn46gmu4juc9tau2w45l9cnf8n0797nmmtkdv7jscv88ra"                     && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET -o json --broadcast-mode block -y --gas=600000 --chain-id=space-pussy-1 --node=https://rpc.space-pussy-1.cybernode.ai:443
{
    "code": 0,
    "codespace": "",
    "data": "0A260A242F636F736D7761736D2E7761736D2E76312E4D736745786563757465436F6E7472616374",
    "gas_used": "400125",
    "gas_wanted": "600000",
    "height": "1444585",
    "info": "",
    "logs": [
        {
            "events": [
                {
                    "attributes": [
                        {
                            "key": "particleFrom",
                            "value": "Qmb6ckJ1zaYx2htqzsgJSK4D7QRbJesPtaL9zYHq5EVnkg"
                        },
                        {
                           

KeyboardInterrupt: 

In [ ]:
parse_contract_execution_json(create_passport_json)

### Proof (Add) Address to Passports

In [ ]:
def proof_address(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    return execute_bash(
            f'''REGISTER='{{"proof_address":{{"address":"{claim_row[network + "_address"]}", "nickname":"{claim_row["nickname"]}", "signature":"{claim_row[network + "_message_signature"]}"}}}}' \
                && CONTRACT="{passport_contract_address}" \
                && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET -o json --broadcast-mode block -y --gas=400000 --chain-id={NETWORK} --node={NODE_URL}''',
            display_data=display_data)

for index, row in tqdm(claims_df.iterrows()):
    if index == 1:
        proof_ethereum_address_json, _ = proof_address(row, display_data=True)
        proof_cosmos_address_json, _ = proof_address(row, network='cosmos', display_data=True)
    else:
        proof_address(row)
        proof_address(row, network='cosmos', display_data=True)

In [ ]:
parse_contract_execution_json(proof_ethereum_address_json)

In [ ]:
parse_contract_execution_json(proof_cosmos_address_json)

### Generate Proofs and Claim

In [ ]:
def claim(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    return execute_bash(
                f'''CLAIM='{{"claim":{{"nickname":"{claim_row['nickname']}", "gift_claiming_address":"{claim_row[network + "_address"]}", "gift_amount":"{claim_row.amount}", "proof":{claim_row[network + "_proof"]}}}}}' \
                    && cyber tx wasm execute {gift_contract_address} "$CLAIM" --from $WALLET -o json --broadcast-mode block -y --gas 400000 --chain-id={NETWORK} --node={NODE_URL}''',
                display_data=display_data)


for index, row in tqdm(claims_df.iterrows()):
    if index == 0:
        claim_ethereum_json, _ = claim(row, display_data=True)
        claim_cosmos_json, _ = claim(row, network='cosmos', display_data=True)
    else:
        claim_ethereum_json_2, _ = claim(row)
        claim_cosmos_json_2, _ = claim(row, network='cosmos')

In [ ]:
parse_contract_execution_json(claim_ethereum_json)

In [ ]:
parse_contract_execution_json(claim_cosmos_json)

In [ ]:
parse_contract_execution_json(claim_ethereum_json_2)

In [ ]:
parse_contract_execution_json(claim_cosmos_json_2)

### Release Gift

In [ ]:
def release(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    return execute_bash(
                f'''CLAIM='{{"release":{{"gift_address":"{claim_row[network + "_address"]}"}}}}' \
                    && cyber tx wasm execute {gift_contract_address} "$CLAIM" --from $WALLET -o json --broadcast-mode block -y --gas 400000 --chain-id={NETWORK} --node={NODE_URL}''',
                display_data=display_data)

for index, row in tqdm(claims_df.iterrows()):
    if index == 0:
        release_ethereum_json, _ = release(row, display_data=True)
        release_cosmos_json, _ = release(row, network='cosmos', display_data=True)
    else:
        release(row)
        release(row, network='cosmos')


In [ ]:
parse_contract_execution_json(release_ethereum_json)

In [ ]:
parse_contract_execution_json(release_cosmos_json)

### Add tests for Passport contract
#### create_passport +
#### update_name
#### update_avatar
#### proof_address +
#### remove_address
#### set_minter
#### set_owner
#### set_active
#### set_subspaces
#### transfer_nft
#### send_nft
#### mint
#### burn
#### approve
#### approve_all
#### revoke
#### revoke_all
#### Expirations
##### at_height
##### at_time
##### never

### Add tests for Gift contract
#### update_owner
#### update_passport_addr
#### update_target
#### register_merkle_root +
#### claim +
#### release +

### Add tests for Subspace contract
#### update_owner
#### update_executer +